In [2]:
import tabula
import PyPDF2

In [21]:


def extract_tables_from_pdf(file_path):
    # Open the PDF file
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages in the PDF
    num_pages = len(pdf_reader.pages)

    # Initialize an empty list to store the extracted tables
    tables = []

    # Iterate over each page in the PDF
    for page_num in range(num_pages):
        # Extract the tables from the current page
        page_tables = tabula.read_pdf(file_path, pages=page_num+1, multiple_tables=True)

        # Add the extracted tables to the list
        tables.extend(page_tables)

    # Close the PDF file
    pdf_file.close()

    return tables

# Example usage:
caminho_pdf = './files/old_pdfs/CCI1.088_2024.pdf' # './files/old_pdfs/CCI1.088_2024.pdf'
tables = extract_tables_from_pdf(caminho_pdf)

# Print the extracted tables
for table in tables:
    display(table)

,Data,Descrição,Responsável
0,Até 31/10/2024,Envio da planilha Plano de Ação 2025 por meio ...,Cooperativas centrais
1,Até 5/11/2024,Análiseevalidaçãodasplanilhasrecebidasno\rPort...,CCS


In [22]:
import fitz

doc = fitz.open('./files/old_pdfs/CCI1.088_2024.pdf')
for page in doc:
    tables = page.get_tables()
    for table in tables:
        print(table)

AttributeError: 'Page' object has no attribute 'get_tables'

In [1]:
import ollama

In [12]:
from ollama import chat
from ollama import AsyncClient

with open('./files/pdfs/PDF.txt', 'r', encoding='utf-8') as f:
    context = f.read()
    messages = [
    {
        'role': 'user',
        'content': f'Use o texto abaixo para responder a questão \"De acordo com o texto, qual é a classificação da demanda? Responda de forma curta e concisa.\"\n\n###TEXTO###\n\n```{context}```',
    },
    ]

    output = ''
    async for part in await AsyncClient().chat(
        'gemma2:2b-instruct-q4_K_M',
        messages=messages,
        stream=True, keep_alive='1h',
        options={"temperature": 0}
    ):print(part['message']['content'], end='', flush=True)
    

This document outlines the functionalities available within the Sicoob Partner Plus platform for managing non-cooperative entities. Let's break down the information provided:

**1.  CPF/CNPJ Field Updates:**

* **Objective:** This functionality aims to standardize the fields "CPF" and "Nome" (Name) in the list of non-cooperative entities.
* **Path:** The path to access this functionality is through the "Simulação não associado" (Non-Associative Simulation) section on the Sicoob Partner Plus platform.

**2.  Classifications for Non-Cooperatives:**

* **Objective:** This function allows users to classify non-cooperative entities based on their type.
* **Path:** The path is through the "Simulação não associado" (Non-Associative Simulation) section of the Sicoob Partner Plus platform. 

**3.  New Non-Cooperative Registration:**

* **Objective:** This functionality enables users to register new non-cooperative entities.
* **Path:** Users can access this functionality through the "Simulação 